# Ансамблирование

Теперь решаем задачу регрессии - предскажем цены на недвижимость. Использовать датасет www.kaggle.com...iques/data (train.csv)
Построить случайный лес, вывести важность признаков
Обучить стекинг как минимум 3х моделей, использовать хотя бы 1 линейную модель и 1 нелинейную

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from matplotlib import pyplot as plt
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn import svm
from sklearn.linear_model import RidgeCV

In [2]:
data = pd.read_csv('train.csv')

Открыла описание переменных, решила руками отобрать по-моему мнению те переменные, от которых может зависеть цена недвижимости

In [3]:
selectedColumns = data[['MSSubClass', 'MSZoning', 'LotArea', 'HouseStyle', 'OverallQual', 'YearBuilt', 'GrLivArea', '2ndFlrSF', 'TotalBsmtSF', 'BsmtFinSF1', 'SaleCondition', 'ExterQual']]
X = pd.get_dummies(selectedColumns, columns = ['MSSubClass', 'MSZoning', 'HouseStyle','SaleCondition', 'ExterQual'])
y = pd.Series(data = data['SalePrice'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Построим случайный лес, качество не хуже, чем было на уроке

In [4]:
clf_rf = RandomForestRegressor()
clf_rf.fit(X_train, y_train)

RandomForestRegressor()

In [5]:
clf_rf.score(X_train, y_train)

0.9742740250222833

In [6]:
clf_rf.score(X_test, y_test)

0.8977118156833898

Обучим стекинг с моделями: LinearRegression, DecisionTreeRegressor, SVR. В DecisionTreeRegressor и SVR укажем параметры

In [7]:
Regressor = StackingRegressor(
    [
        ('LinearRegression', LinearRegression()),
        ('DecisionTreeRegressor', DecisionTreeRegressor(max_depth=8, min_samples_leaf=9, criterion='mae')),
        ('SVR', svm.SVR(kernel="linear", gamma='auto'))
    ], RidgeCV())

In [8]:
Regressor.fit(X_train, y_train)

StackingRegressor(estimators=[('LinearRegression', LinearRegression()),
                              ('DecisionTreeRegressor',
                               DecisionTreeRegressor(criterion='mae',
                                                     max_depth=8,
                                                     min_samples_leaf=9)),
                              ('SVR', SVR(gamma='auto', kernel='linear'))],
                  final_estimator=RidgeCV(alphas=array([ 0.1,  1. , 10. ])))

Качество хуже предыдущего примера и хуже того, что было разобранно на уроке. Ниже проверим с теми же моделями, что и на уроке, только у деревьев укажем параметры

In [9]:
print(f'Score on train data {Regressor.score(X_train, y_train)}')
print(f'Score on test data {Regressor.score(X_test, y_test)}')

for i in Regressor.named_estimators:
    print(f'Score on train data with model {i} {Regressor.named_estimators_[i].score(X_train, y_train)}')
    print(f'Score on test data with model {i} {Regressor.named_estimators_[i].score(X_test, y_test)}')

Score on train data 0.8334117711294041
Score on test data 0.8661187829393344
Score on train data with model LinearRegression 0.7939450162828887
Score on test data with model LinearRegression 0.8345167287214925
Score on train data with model DecisionTreeRegressor 0.8298829720946995
Score on test data with model DecisionTreeRegressor 0.846658331951861
Score on train data with model SVR 0.673095694196723
Score on test data with model SVR 0.7539811034605979


In [10]:
Regressor1 = StackingRegressor(
    [
        ('LinearRegression', LinearRegression()),
        ('KNeighborsRegressor', KNeighborsRegressor()),
        ('DecisionTree', DecisionTreeRegressor(max_depth=8, min_samples_leaf=9, criterion='mae'))
    ], RidgeCV())

In [11]:
Regressor1.fit(X_train, y_train)

StackingRegressor(estimators=[('LinearRegression', LinearRegression()),
                              ('KNeighborsRegressor', KNeighborsRegressor()),
                              ('DecisionTree',
                               DecisionTreeRegressor(criterion='mae',
                                                     max_depth=8,
                                                     min_samples_leaf=9))],
                  final_estimator=RidgeCV(alphas=array([ 0.1,  1. , 10. ])))

In [12]:
print(f'Score on train data {Regressor1.score(X_train, y_train)}')
print(f'Score on test data {Regressor1.score(X_test, y_test)}')

for i in Regressor1.named_estimators:
    print(f'Score on train data with model {i} {Regressor1.named_estimators_[i].score(X_train, y_train)}')
    print(f'Score on test data with model {i} {Regressor1.named_estimators_[i].score(X_test, y_test)}')

Score on train data 0.8533099053192948
Score on test data 0.8680080520224095
Score on train data with model LinearRegression 0.7939450162828887
Score on test data with model LinearRegression 0.8345167287214925
Score on train data with model KNeighborsRegressor 0.7241940903770001
Score on test data with model KNeighborsRegressor 0.6678950902881424
Score on train data with model DecisionTree 0.8298829720946995
Score on test data with model DecisionTree 0.846658331951861


Качество не намного выше. Обработаем перменные так же как на уроке и сравним результаты

In [14]:
data_train, data_test, y_train, y_test = train_test_split(data.drop(columns='SalePrice'), data['SalePrice'], test_size=0.30, random_state=42)

In [15]:
categorials = list(data.dtypes[data.dtypes == object].index)
categorials_nunique = data_train[categorials].nunique()
data[categorials] = data[categorials].fillna('nan')

In [16]:
integer = [f for f in data if f not in (categorials + ['ID', 'SalePrice'])]

In [17]:
dummy_train = pd.get_dummies(data_train[categorials], columns=categorials)
dummy_test = pd.get_dummies(data_test[categorials], columns=categorials)
dummy_cols = list(set(dummy_train) & set(dummy_test))
dummy_train = dummy_train[dummy_cols]
dummy_test = dummy_test[dummy_cols]
X_train = pd.concat([data_train[integer].fillna(-999), dummy_train], axis=1)
X_test = pd.concat([data_test[integer].fillna(-999), dummy_test], axis=1)

In [24]:
Regressor_new = StackingRegressor(
    [
        ('LinearRegression', LinearRegression()),
        ('DecisionTreeRegressor', DecisionTreeRegressor(max_depth=8, min_samples_leaf=9, criterion='mae')),
        ('SVR', svm.SVR(kernel="linear", gamma='auto'))
    ], RidgeCV())

In [25]:
Regressor_new.fit(X_train, y_train)

StackingRegressor(estimators=[('LinearRegression', LinearRegression()),
                              ('DecisionTreeRegressor',
                               DecisionTreeRegressor(criterion='mae',
                                                     max_depth=8,
                                                     min_samples_leaf=9)),
                              ('SVR', SVR(gamma='auto', kernel='linear'))],
                  final_estimator=RidgeCV(alphas=array([ 0.1,  1. , 10. ])))

In [20]:
print(f'Score on train data {Regressor_new.score(X_train, y_train)}')
print(f'Score on test data {Regressor_new.score(X_test, y_test)}')

for i in Regressor_new.named_estimators:
    print(f'Score on train data with model {i} {Regressor_new.named_estimators_[i].score(X_train, y_train)}')
    print(f'Score on test data with model {i} {Regressor_new.named_estimators_[i].score(X_test, y_test)}')

Score on train data 0.8906782884098084
Score on test data 0.8813911348887876
Score on train data with model LinearRegression 0.9221085403562431
Score on test data with model LinearRegression 0.8533836050579311
Score on train data with model DecisionTreeRegressor 0.8469881301763209
Score on test data with model DecisionTreeRegressor 0.8240383394873092
Score on train data with model SVR 0.7227878953046374
Score on test data with model SVR 0.7904194892662872


In [21]:
Regressor1_new = StackingRegressor(
    [
        ('LinearRegression', LinearRegression()),
        ('KNeighborsRegressor', KNeighborsRegressor()),
        ('DecisionTree', DecisionTreeRegressor(max_depth=8, min_samples_leaf=9, criterion='mae'))
    ], RidgeCV())

In [22]:
Regressor1_new.fit(X_train, y_train)

StackingRegressor(estimators=[('LinearRegression', LinearRegression()),
                              ('KNeighborsRegressor', KNeighborsRegressor()),
                              ('DecisionTree',
                               DecisionTreeRegressor(criterion='mae',
                                                     max_depth=8,
                                                     min_samples_leaf=9))],
                  final_estimator=RidgeCV(alphas=array([ 0.1,  1. , 10. ])))

In [26]:
print(f'Score on train data {Regressor1_new.score(X_train, y_train)}')
print(f'Score on test data {Regressor1_new.score(X_test, y_test)}')

for i in Regressor1_new.named_estimators:
    print(f'Score on train data with model {i} {Regressor1_new.named_estimators_[i].score(X_train, y_train)}')
    print(f'Score on test data with model {i} {Regressor1_new.named_estimators_[i].score(X_test, y_test)}')

NameError: name 'Regressor1_new' is not defined

Качество стало выше, но не выше того, которое было получено на уроке